In [ ]:
import os

In [2]:
!pwd

/home/jovyan/notebooks/executions


In [3]:
import sys
sys.path.append('/home/jovyan/')

In [4]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration
from neat.configuration import get_configuration, set_configuration
from neat.evaluation.evaluation_engine import evaluate_genome, get_dataset
from neat.genome import Genome
import os
from neat.loss.vi_loss import get_loss

In [5]:
# dir(neat)

In [57]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

### Configuration

In [95]:
execution_id = 'cb3a361c-922a-4fd1-9368-1fc341cc9d1f'
execution_id = '624c3966-15ec-46d0-ab64-506b994fa3c8'
execution_id = '6bfafd62-1590-4f24-93c6-7703b93738fd'
execution_id = '249d73e6-476e-4a20-9c7c-3726e2fefb0b'  # gen 10

execution_id = '527d9ac9-ec6f-4ba0-a842-253bf0ec06bc'  # gen 1
execution_id = 'fe6d383c-cab7-4939-83fa-cdf387ab0744'  # new initialization
DATASET = 'mnist_downsampled'
ALGORITHM_VERSION = 'bayes-neat'
CORRELATION_ID = 'test'

In [96]:
# execution_id = '2014fc1f-50e0-42b8-800e-69102967c77f' # -0.22
# DATASET = 'mnist_binary'
# ALGORITHM_VERSION = 'bayes-neat'
# CORRELATION_ID = 'test'

In [97]:
report_repository = ReportRepository.create(project='neuro-evolution', logs_path=LOGS_PATH)
report = report_repository.get_report(algorithm_version=ALGORITHM_VERSION,
                                      dataset=DATASET,
                                      correlation_id=CORRELATION_ID,
                                      execution_id=execution_id)

In [98]:
genome_dict = report.data['best_individual']
best_individual_fitness = report.data['best_individual_fitness']
print(f'Fitness of best individual: {best_individual_fitness}')

genome = Genome.from_dict(genome_dict)
config = genome.genome_config
set_configuration(config)

Fitness of best individual: -2.4978134632110596


TypeError: unhashable type: 'dict'

In [99]:
genome_dict

{'key': 217,
 'id': '98d3adc7-ff20-4d7a-b7dd-c5e6ff191adb',
 'n_input': 256,
 'n_output': 10,
 'output_nodes_keys': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'input_nodes_keys': [-1,
  -2,
  -3,
  -4,
  -5,
  -6,
  -7,
  -8,
  -9,
  -10,
  -11,
  -12,
  -13,
  -14,
  -15,
  -16,
  -17,
  -18,
  -19,
  -20,
  -21,
  -22,
  -23,
  -24,
  -25,
  -26,
  -27,
  -28,
  -29,
  -30,
  -31,
  -32,
  -33,
  -34,
  -35,
  -36,
  -37,
  -38,
  -39,
  -40,
  -41,
  -42,
  -43,
  -44,
  -45,
  -46,
  -47,
  -48,
  -49,
  -50,
  -51,
  -52,
  -53,
  -54,
  -55,
  -56,
  -57,
  -58,
  -59,
  -60,
  -61,
  -62,
  -63,
  -64,
  -65,
  -66,
  -67,
  -68,
  -69,
  -70,
  -71,
  -72,
  -73,
  -74,
  -75,
  -76,
  -77,
  -78,
  -79,
  -80,
  -81,
  -82,
  -83,
  -84,
  -85,
  -86,
  -87,
  -88,
  -89,
  -90,
  -91,
  -92,
  -93,
  -94,
  -95,
  -96,
  -97,
  -98,
  -99,
  -100,
  -101,
  -102,
  -103,
  -104,
  -105,
  -106,
  -107,
  -108,
  -109,
  -110,
  -111,
  -112,
  -113,
  -114,
  -115,
  -116,
  -117,
  -1

In [62]:
loss = get_loss(problem_type=config.problem_type)
dataset = get_dataset(config.dataset, testing=True)

In [63]:
x, y_true, y_pred, loss_value = evaluate_genome(genome=genome,
                                                    dataset=dataset,
                                                    loss=loss,
                                                    problem_type=config.problem_type,
                                                    beta_type=config.beta_type,
                                                    batch_size=config.batch_size,
                                                    n_samples=1,
                                                    is_gpu=config.is_gpu,
                                                    return_all=True)
print(y_true)
y_pred

tensor([7, 2, 1,  ..., 4, 5, 6])


tensor([[0.4928, 1.4135, 0.5382,  ..., 0.5254, 0.9140, 1.2830],
        [0.4928, 1.4135, 0.5382,  ..., 0.5254, 0.9140, 1.2830],
        [0.4928, 1.4135, 0.5382,  ..., 0.5254, 0.9140, 1.2830],
        ...,
        [0.4928, 1.3018, 0.5382,  ..., 0.5254, 0.9140, 1.2830],
        [0.4928, 1.4135, 0.5382,  ..., 0.5254, 0.9140, 1.2830],
        [0.4927, 0.8056, 0.5382,  ..., 0.5254, 0.9140, 1.2830]])

In [64]:
loss_value

2.334423065185547

In [65]:
x.shape

torch.Size([10000, 256])

In [66]:
y_true.shape

torch.Size([10000])

In [67]:
y_pred.shape

torch.Size([10000, 10])

In [68]:
for connection in genome_dict['connection_genes'].values():
#     connection['_weight_std'] = 0.000000001
    value = connection['_weight_std']
    print(value)

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001


In [69]:
for connection in genome_dict['node_genes'].values():
    value = connection['_bias_std']
#     connection['_bias_std'] = 0.000000001
    print(value)

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001


## Show graph

In [77]:
from neat.plotting.plot_network import plot_genome_network

In [82]:
display(plot_genome_network(genome))

None

In [118]:
genome_dict['node_genes']

{'0': {'key': 0,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': 0.8057251123197042,
  '_bias_std': 0.001},
 '1': {'key': 1,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': 1.0,
  '_bias_std': 0.001},
 '2': {'key': 2,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': 1.0,
  '_bias_std': 0.001},
 '3': {'key': 3,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': -1.0,
  '_bias_std': 0.001},
 '4': {'key': 4,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': -0.34778403774296407,
  '_bias_std': 0.001},
 '5': {'key': 5,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': -0.3556845272430937,
  '_bias_std': 0.001},
 '6': {'key': 6,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': 1.0,
  '_bias_std': 0.001},
 '7': {'key': 7,
  'activation': 'tanh',
  'aggregation': 'sum',
  '_bias_mean': -0.1276673725763255,
  '_bias_std': 0.001},
 '8': {'key': 8,
  'activation': 'tanh',
  'aggregation': 'sum'

In [9]:
config_file = 'mnist_downsampled'
config = create_configuration(filename=f'/{config_file}.json')

In [13]:
# TODO: better mechanism for override
config.pop_size = 20
# config.n_samples = 50
#
config.n_generations = 300
config.node_add_prob = 0.5
config.n_output = 2
config.n_processes = 2


ALGORITHM_VERSION = 'bayes-neat'
DATASET = 'mnist_binary'
# CORRELATION_ID = 'parameters_grid'
CORRELATION_ID = 'solve_binary'

In [14]:
report_repository = ReportRepository.create(project='neuro-evolution', logs_path=LOGS_PATH)
notifier = SlackNotifier.create(channel='batch-jobs')

In [15]:
report = EvolutionReport(report_repository=report_repository,
                         algorithm_version=ALGORITHM_VERSION,
                         dataset=DATASET,
                         correlation_id=CORRELATION_ID)

In [16]:
population_engine = PopulationEngine(stagnation_engine=Stagnation())
speciation_engine = SpeciationEngine()
evaluation_engine = EvaluationStochasticEngine()

GETTING DATASET
GETTING DATASET


In [17]:
evaluation_engine.n_processes

2

### Initialize

In [18]:
end_condition = 'normal'
# try:
# initialize population
population = population_engine.initialize_population()
speciation_engine.speciate(population, generation=0)

population = evaluation_engine.evaluate(population=population)

# report
report.report_new_generation(generation=0,
                                  population=population,
                                  species=speciation_engine.species)

2020-03-19 18:15:03,708 - INFO - Generation 0. Best fitness: -198.308. N-Parameters Best: 8
2020-03-19 18:15:03,709 - INFO -                          Mean fitness: -199.773. Mean N-Parameters: 8.0
2020-03-19 18:15:03,710 - INFO -     New best individual (20) found with fitness -198.308
GETTING DATASET
Confusion Matrix:
[[48435 49565]
 [56598 56902]]
Accuracy: 49.8047281323877 %


In [22]:
def run_generation(generation, population):
    
    # create new generation's population
    population = population_engine.reproduce(species=speciation_engine.species,
                                             pop_size=population_engine.pop_size,
                                             generation=generation)
    # create new species based on new population
    speciation_engine.speciate(population, generation=generation)

    # evaluate
    population = evaluation_engine.evaluate(population=population)

    # generation report
    report.report_new_generation(generation=generation,
                                 population=population,
                                 species=speciation_engine.species)
    generation += 1
    return population

In [23]:
generation = 1

In [27]:
generation += 1
population = run_generation(generation, population)

2020-03-20 09:39:33,083 - INFO - Generation 3. Best fitness: -0.956. N-Parameters Best: 24
2020-03-20 09:39:33,083 - INFO -                          Mean fitness: -21.167. Mean N-Parameters: 22.8


### Check evaluation results

In [31]:
from neat.evaluation.evaluate_simple import evaluate_genome
from neat.evaluation.utils import get_dataset
from neat.loss.vi_loss import get_loss, get_beta

In [29]:
testing = False

In [33]:
dataset = get_dataset(config.dataset, testing=testing)
loss = get_loss(problem_type=config.problem_type)

GETTING DATASET
